In [ ]:
!pip install langchain
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install  bitsandbytes
!pip install openai
!pip install transformers --upgrade
!pip install pypdf

In [ ]:
import os
import openai
import torch
from langchain.document_loaders import PyPDFLoader
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

In [ ]:
#Custom_Model
model_id = "google/flan-t5-small"
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'HUGGINGFACEHUB_API_TOKEN'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto')

In [ ]:
pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=128)

In [ ]:
local_llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
prompt = PromptTemplate(
    input_variables= ["text"],
    template= '''
You are a experienced and highly talented and creative resume summary helper. Parse the resume contents and give summary for the contents. IF unable to parse the content display not parsable and if encountered any images try to read the text from images or else neglect it.

If you not able to parse the content explain what was the reason to unable to parse it like unable to understand the key or symbol etc



sample example of response for a sample resume given below:

Resume Summary =
{
  name: "Rakesh Kamma",
  "contact": {
    "phone": "+1 425-362-5163",
    "email": "kammarakesh2000@gmail.com"
  },
  "education": [
    {
      "degree": "Master’s",
      "major": "Computer Science",
      "school": "Northern Illinois University(NIU)",
      "location": "Chicago, U.S.A",
      "dates": "Jan 2022 - Present",
      "CGPA": "3.7"
    },
    {
      "degree": "Bachelor of Technology",
      "major": "Computer Science",
      "school": "Amrita School Of Engineering",
      "location": "Bangalore, India",
      "dates": "July 2017 - June 2021",
      "CGPA": "7.91"
    }
  ],
  "skills": {
    "languages": ["SQL", "Python", "R", "JAVA", "JavaScript", "Shell Scripting", "HTML5", "CSS", "UNIX", "C#"],
    "databases": ["MySql", "PostgreSql", "MongoDB"],
    "libraries": ["NumPy", "Pandas", "Scikit", "TensorFlow", "Keras", "NLTK"],
    "frameworks": ["FLask", "FastApi", "Django", "Hadoop", "Spark"],
    "tools": ["DataBricks", "PowerBI", "Snowflake", "PySpark", "Informatica cloud", "Microsoft Visual Studio", "GIT", "Docker"],
    "certifications": ["Microsoft PowerBI", "Microsoft Python Data Science", "AWS cloud practitioner"],
    "protocols": ["Hippa", "HL7", "Kafka"],
    "hard_skills": ["Data Structures", "Programming logic", "OOPs"],
    "soft_skills": ["Leadership", "Event Management", "Time Management"]
  },
  "experience": [
    {
      "title": "Teaching Assistant",
      "dates": "Aug 2022 - May 2023",
      "employer": "Northern Illinois University",
      "responsibilities": [
        "Assisted professors in teaching statistical tools to undergraduates in all levels of math courses."
      ]
    },
    {
      "title": "Research Assistant",
      "dates": "May 2022 - Jan 2023",
      "employer": "Northern Illinois University",
      "responsibilities": [
        "Investigated vessel travel patterns and produced vessel traffic summaries using Python scripts.",
        "Constructed knowledge graphs to represent the US tax code, streamlining the process using automation tools."
      ]
    },
    {
      "title": "Software Engineer",
      "dates": "Mar 2021 - Jan 2022",
      "employer": "Knowledge Lens",
      "responsibilities": [
        "Optimized programs and modules for the ILENS project using Python, Fast API, Flask, MongoDB, and PostgreSQL.",
        "Developed a basic ETL process, resulting in cost savings and faster data processing using Spark in Python.",
        "Implemented precision and efficacy controls and programs for data processing."
      ]
    },
    {
      "title": "Internship",
      "dates": "Jan 2021 - Mar 2021",
      "employer": "Knowledge Lens",
      "responsibilities": [
        "Automated consultant onboarding through the Onboarding Portal Management project using Fast API and PostgreSQL."
      ]
    },
    {
      "title": "Placement Co-Ordinator",
      "dates": "Jun 2019 - Dec 2020",
      "employer": "Amrita School of Engineering",
      "responsibilities": [
        "Developed, implemented, and managed job placement and work-based learning activities for students."
      ]
    }
  ],
  "personal_projects": [
    {
      "title": "Student Information System",
      "description": "A desktop application to manage student records using C .NET Framework, Microsoft SQL Server, and Visual Studio."
    },
    {
      "title": "Drowsiness Detection",
      "description": "Model using Haar-Cascade classifier and NLP to detect drowsiness based on facial expressions."
    },
    {
      "title": "Real-Time Fruit Detection",
      "description": "Model using YOLOv3 algorithm to detect fruit quality for food grading based on images."
    },
    {
      "title": "Classification of Images Using R",
      "description": "Deep learning-based image classifier using R language and convolutional neural networks."
    },
    {
      "title": "Restaurant Management System",
      "description": "Web application for meal preparation and restaurant menu orders using HTML, CSS, PHP, MYSQL, and Xampp server."
    },
    {
      "title": "Cab Rental Project",
      "description": "Platform for ride-sharing and car rental using web technology to connect travelers."
    }
  ],
  "honors_awards": [
    "1st place in the International Mathematics Olympiad (IMO) test - 2015",
    "2nd place in the University SLAC Competition - 2019",
    "Best Intern of Knowledge Lens - 2021"
  ]

  Summarize the

  {text}


  from the given loaded data from pdf according to the above given json format and output the text in terms of json format



}'''
)

In [ ]:
prompt = '''
You are a experienced and highly talented and creative resume summary helper. Parse the resume contents and give summary for the contents. IF unable to parse the content display not parsable and if encountered any images try to read the text from images or else neglect it.

If you not able to parse the content explain what was the reason to unable to parse it like unable to understand the key or symbol etc



sample example of response for a sample resume given below:

Resume Summary =
{
  name: "Rakesh Kamma",
  "contact": {
    "phone": "+1 425-362-5163",
    "email": "kammarakesh2000@gmail.com"
  },
  "education": [
    {
      "degree": "Master’s",
      "major": "Computer Science",
      "school": "Northern Illinois University(NIU)",
      "location": "Chicago, U.S.A",
      "dates": "Jan 2022 - Present",
      "CGPA": "3.7"
    },
    {
      "degree": "Bachelor of Technology",
      "major": "Computer Science",
      "school": "Amrita School Of Engineering",
      "location": "Bangalore, India",
      "dates": "July 2017 - June 2021",
      "CGPA": "7.91"
    }
  ],
  "skills": {
    "languages": ["SQL", "Python", "R", "JAVA", "JavaScript", "Shell Scripting", "HTML5", "CSS", "UNIX", "C#"],
    "databases": ["MySql", "PostgreSql", "MongoDB"],
    "libraries": ["NumPy", "Pandas", "Scikit", "TensorFlow", "Keras", "NLTK"],
    "frameworks": ["FLask", "FastApi", "Django", "Hadoop", "Spark"],
    "tools": ["DataBricks", "PowerBI", "Snowflake", "PySpark", "Informatica cloud", "Microsoft Visual Studio", "GIT", "Docker"],
    "certifications": ["Microsoft PowerBI", "Microsoft Python Data Science", "AWS cloud practitioner"],
    "protocols": ["Hippa", "HL7", "Kafka"],
    "hard_skills": ["Data Structures", "Programming logic", "OOPs"],
    "soft_skills": ["Leadership", "Event Management", "Time Management"]
  },
  "experience": [
    {
      "title": "Teaching Assistant",
      "dates": "Aug 2022 - May 2023",
      "employer": "Northern Illinois University",
      "responsibilities": [
        "Assisted professors in teaching statistical tools to undergraduates in all levels of math courses."
      ]
    },
    {
      "title": "Research Assistant",
      "dates": "May 2022 - Jan 2023",
      "employer": "Northern Illinois University",
      "responsibilities": [
        "Investigated vessel travel patterns and produced vessel traffic summaries using Python scripts.",
        "Constructed knowledge graphs to represent the US tax code, streamlining the process using automation tools."
      ]
    },
    {
      "title": "Software Engineer",
      "dates": "Mar 2021 - Jan 2022",
      "employer": "Knowledge Lens",
      "responsibilities": [
        "Optimized programs and modules for the ILENS project using Python, Fast API, Flask, MongoDB, and PostgreSQL.",
        "Developed a basic ETL process, resulting in cost savings and faster data processing using Spark in Python.",
        "Implemented precision and efficacy controls and programs for data processing."
      ]
    },
    {
      "title": "Internship",
      "dates": "Jan 2021 - Mar 2021",
      "employer": "Knowledge Lens",
      "responsibilities": [
        "Automated consultant onboarding through the Onboarding Portal Management project using Fast API and PostgreSQL."
      ]
    },
    {
      "title": "Placement Co-Ordinator",
      "dates": "Jun 2019 - Dec 2020",
      "employer": "Amrita School of Engineering",
      "responsibilities": [
        "Developed, implemented, and managed job placement and work-based learning activities for students."
      ]
    }
  ],
  "personal_projects": [
    {
      "title": "Student Information System",
      "description": "A desktop application to manage student records using C .NET Framework, Microsoft SQL Server, and Visual Studio."
    },
    {
      "title": "Drowsiness Detection",
      "description": "Model using Haar-Cascade classifier and NLP to detect drowsiness based on facial expressions."
    },
    {
      "title": "Real-Time Fruit Detection",
      "description": "Model using YOLOv3 algorithm to detect fruit quality for food grading based on images."
    },
    {
      "title": "Classification of Images Using R",
      "description": "Deep learning-based image classifier using R language and convolutional neural networks."
    },
    {
      "title": "Restaurant Management System",
      "description": "Web application for meal preparation and restaurant menu orders using HTML, CSS, PHP, MYSQL, and Xampp server."
    },
    {
      "title": "Cab Rental Project",
      "description": "Platform for ride-sharing and car rental using web technology to connect travelers."
    }
  ],
  "honors_awards": [
    "1st place in the International Mathematics Olympiad (IMO) test - 2015",
    "2nd place in the University SLAC Competition - 2019",
    "Best Intern of Knowledge Lens - 2021"
  ]

  Summarize the content from the given loaded data from pdf according to the above given json format and output the text in terms of json format


}'''


In [ ]:
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
from langchain.vectorstores import Chroma

In [ ]:
chain = LLMChain(llm=local_llm,prompt = prompt)

In [ ]:
loader = PyPDFLoader("Your resume")
pdf_doc = loader.load()


In [ ]:
chain.run(pdf_doc)